In [3]:
import pandas as pd
import numpy as np
import extraction as e

In [51]:
# Combine the data and drop unnecessary columns

# List the years included in the data
years = ['18','19','20','21','22']
# List of the simple column names
new_cols = ['spotify_id','artist_id','track_name','album_name','artist_name','date', 'duration','pop', 'genres','danceability','energy','key','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
# Initialize empty dataframes to append all csv files together for easier parsing 
tophitsdata = pd.DataFrame()
mysongsdata = pd.DataFrame()
# Clean data for each year
for year in years:
    tophits = pd.read_csv(f"raw_datasets/tophits20{year}.csv")
    mysongs = pd.read_csv(f"raw_datasets/mysongs20{year}.csv")
    # Drop unnecessary columns
    tophits = tophits.drop(columns=['Added By','Added At','Time Signature','Mode'])
    mysongs = mysongs.drop(columns=['Added By','Added At','Time Signature','Mode'])
    # Rename columns
    tophits.columns = new_cols
    mysongs.columns = new_cols
    # Add "year" column for easier grouping
    tophits['year'] = [f"20{year}"] * tophits.shape[0]
    mysongs['year'] = [f"20{year}"] * mysongs.shape[0]
    # Append csv files together
    tophitsdata = pd.concat([tophitsdata,tophits], ignore_index=True)
    mysongsdata = pd.concat([mysongsdata, mysongs], ignore_index=True)
    
    # Replace NaN genre values with 'none' element
    tophitsdata['genres'] = tophitsdata['genres'].fillna('none')
    mysongsdata['genres'] = mysongsdata['genres'].fillna('none')


# Write one csv file for each dataset
mysongsdata.to_csv("datasets/mysongsdata.csv")
tophitsdata.to_csv("datasets/tophitsdata.csv")

In [52]:
# Divide artists for both datasets
songs_rs = e.divide_artists(mysongsdata)
songs_rs.to_csv("datasets/mysongsdata_rs.csv")
hits_rs = e.divide_artists(tophitsdata)
hits_rs.to_csv("datasets/tophitsdata_rs.csv")

In [53]:
# Divide genres for both datasets
hits_rrs = e.divide_genres(tophitsdata)
hits_rrs.to_csv("datasets/tophitsdata_rrs.csv")
songs_rrs = e.divide_genres(mysongsdata)
songs_rrs.to_csv("datasets/mysongsdata_rrs.csv")

In [54]:
# Divide artists and genres for both datasets
hits_rrrs = e.divide_all(tophitsdata)
hits_rrrs.to_csv("datasets/tophitsdata_rrrs.csv")
songs_rrrs = e.divide_all(mysongsdata)
songs_rrrs.to_csv("datasets/mysongsdata_rrrs.csv")

In [55]:
# Get all unique artist ids and names
songs_artists = songs_rs[['artist_id','artist_name']]
hits_artists = hits_rs[['artist_id','artist_name']]
all_artists = pd.concat([hits_artists, songs_artists],ignore_index=True)
all_artists = all_artists.drop_duplicates().reset_index(drop=True)

# Fetch and add artist genres
artist_rows = all_artists.to_dict(orient="records")
cur = 1
for row in artist_rows:
    genres = e.get_artist_genres(e.get_token(), row["artist_id"])
    if len(genres) == 0:
        row["genres"] = 'none'
    elif len(genres) == 1:
        row["genres"] = genres[0]
    else:
        row["genres"] = ",".join(genres)
    print(f"Done with line {cur}..")
    cur += 1
    
all_artists = pd.DataFrame(artist_rows)
all_artists.to_csv("datasets/artists_info.csv")

In [56]:
# Get a list of all unique genres in all data
genres = set()
datasets = [tophitsdata['genres'],mysongsdata['genres']]
for data in datasets:
    for k in data:
        if e.is_float(k) == True:
            continue
        if "," in k:
            subset = e.split_element(k)
            for genre in subset:
                genres.add(genre)
        else:
            genres.add(k)
# print(genres)

In [57]:
# Add latin/nonlatin column for tracks to all datasets
all_data = ["mysongsdata.csv","tophitsdata.csv",
            "mysongsdata_rs.csv","tophitsdata_rs.csv",
            "mysongsdata_rrs.csv","tophitsdata_rrs.csv",
            "mysongsdata_rrrs.csv","tophitsdata_rrrs.csv",
            "artists_info.csv"]
for dataset in all_data:
    ds = pd.read_csv(f"datasets/{dataset}").drop(columns="Unnamed: 0")
    rows = ds.to_dict(orient="records")                 
    for row in rows:
        row["is_latin"] = e.is_latin(row)
    ds = pd.DataFrame(rows)
    ds.to_csv(f"datasets/{dataset}")

In [58]:
# Fetch latin and unknown artists
ai = pd.read_csv('datasets/artists_info.csv').drop(columns="Unnamed: 0")
latin_artists = list(ai[ai["is_latin"]=='latin']["artist_name"])
nonlatin_artists = list(ai[ai["is_latin"]=='nonlatin']["artist_name"])
# Add artist's latin status to datasets
all_data = ["mysongsdata_rs.csv", "tophitsdata_rs.csv",
            "mysongsdata_rrrs.csv", "tophitsdata_rrrs.csv"]
for dataset in all_data:
    ds = pd.read_csv(f"datasets/{dataset}").drop(columns="Unnamed: 0")
    rows = ds.to_dict(orient="records")                 
    for row in rows:
        row["artist_latin_status"] = "unknown"
        for artist in nonlatin_artists:
            if artist in row["artist_name"]:
                row["artist_latin_status"] = "nonlatin"
        for artist in latin_artists:
            if artist in row["artist_name"]:
                row["artist_latin_status"] = "latin"
    ds = pd.DataFrame(rows)
    ds.to_csv(f"datasets/{dataset}")

In [48]:
# Fetch counts for my personal genre progression
final = pd.DataFrame(columns=['genres','genre_counts','year'])
f = pd.read_csv("datasets/mysongsdata_rrs.csv").drop(columns="Unnamed: 0")
latin_genres = e.get_latin_genres()
genres = list(f['genres'].unique())
yrs = list(f['year'].unique())
for yr in yrs:
    count = f[f["year"]==yr].groupby('genres').size()
    temp = pd.DataFrame(count).reset_index().rename(columns={0:'genre_counts'})
    for genre in genres:
        if genre not in temp['genres']:
            temp.loc[len(temp.index)] = [genre, 0]
    temp['year'] = [yr]*len(temp)
    final = pd.concat([final, temp], ignore_index=True)  

rows = final.to_dict(orient="records")                 
for row in rows:
    row["is_latin"] = e.is_latin(row)
final = pd.DataFrame(rows)
final.to_csv("datasets/genre_counts.csv")